###  SQL Keys, Normalization, and ACID Properties


---

### 1. **Introduction to SQL Keys**

**Keys** in SQL are critical to maintaining the integrity of the database by ensuring that each record within a table is uniquely identifiable.

#### **1.1. Primary Key**
- **Definition**: A primary key is a column (or a set of columns) that uniquely identifies each row in a table.
- **Characteristics**:
  - Uniqueness: No two rows can have the same primary key value.
  - Non-null: The primary key field cannot be null.
  
**Example:**

Let’s create a `Students` table where `student_id` is the primary key.

```sql
CREATE DATABASE SchoolDB;
USE SchoolDB;

CREATE TABLE Students (
    student_id INT NOT NULL PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INT,
    grade_level INT
);
```

- Here, `student_id` uniquely identifies each student in the `Students` table.

#### **1.2. Foreign Key**
- **Definition**: A foreign key is a column that creates a relationship between two tables. It refers to the primary key in another table.
- **Purpose**: To enforce referential integrity by ensuring that the value in the foreign key column corresponds to a valid primary key value in the referenced table.

**Example:**

Let’s create a `Classes` table where `teacher_id` references the `Teachers` table.

```sql
CREATE TABLE Teachers (
    teacher_id INT NOT NULL PRIMARY KEY,
    name VARCHAR(50),
    subject VARCHAR(50)
);

CREATE TABLE Classes (
    class_id INT NOT NULL PRIMARY KEY,
    class_name VARCHAR(50),
    teacher_id INT,
    FOREIGN KEY (teacher_id) REFERENCES Teachers(teacher_id)
);
```

- Here, `teacher_id` in the `Classes` table is a foreign key that references `teacher_id` in the `Teachers` table.

#### **1.3. Unique Key**
- **Definition**: A unique key constraint ensures that all the values in a column are different.
- **Difference from Primary Key**: Unlike the primary key, a table can have multiple unique keys, and unique keys can contain null values.

**Example:**

Let’s add a `student_email` column in the `Students` table with a unique constraint.

```sql
ALTER TABLE Students
ADD student_email VARCHAR(100) UNIQUE;
```

- The `student_email` must be unique for every student.

#### **1.4. Composite Key**
- **Definition**: A composite key is a combination of two or more columns in a table that together uniquely identify a row.

**Example:**

Let’s create an `Enrollments` table with a composite key.

```sql
CREATE TABLE Enrollments (
    student_id INT,
    class_id INT,
    enrollment_date DATE,
    PRIMARY KEY (student_id, class_id),
    FOREIGN KEY (student_id) REFERENCES Students(student_id),
    FOREIGN KEY (class_id) REFERENCES Classes(class_id)
);
```

- Here, `student_id` and `class_id` together form a composite key that uniquely identifies each enrollment record.

---



## SQL Normalization

### Introduction to Normalization

Normalization is a process in database design that organizes tables and their relationships to reduce redundancy and dependency. The main goal is to eliminate unnecessary duplication and ensure data is stored logically. This tutorial will take you through the various stages of normalization, explaining each step with examples and detailed SQL queries.

### Step 1: Understanding the Unnormalized Form (UNF)

Before diving into normalization, let's first understand what unnormalized data looks like. In an unnormalized form, all data is stored in a single table without any structure. This might seem convenient initially, but it leads to redundancy and inconsistency.

#### Example:
Consider a table that stores information about students and the courses they are enrolled in:

| StudentID | StudentName | Course1 | Course2 | Course3 |
|-----------|-------------|---------|---------|---------|
| 1         | John Doe    | Math    | Science | NULL    |
| 2         | Jane Smith  | Math    | NULL    | NULL    |
| 3         | Sam Brown   | Science | English | Math    |

#### Problems with UNF:
- **Redundancy**: Course names are repeated.
- **Inconsistency**: If "Math" needs to be renamed, it must be updated in multiple places.
- **Scalability**: Adding more courses requires altering the table structure.

### Step 2: First Normal Form (1NF)

The first step towards normalization is converting data into the First Normal Form (1NF). A table is in 1NF if:
- All columns contain atomic (indivisible) values.
- Each column contains values of a single type.
- Each column has a unique name.
- The order in which data is stored does not matter.

#### Example:
To convert the previous table to 1NF, we separate each course into its own row:

| StudentID | StudentName | Course  |
|-----------|-------------|---------|
| 1         | John Doe    | Math    |
| 1         | John Doe    | Science |
| 2         | Jane Smith  | Math    |
| 3         | Sam Brown   | Science |
| 3         | Sam Brown   | English |
| 3         | Sam Brown   | Math    |

#### SQL Query for 1NF:
```sql
CREATE TABLE Students_Courses (
    StudentID INT,
    StudentName VARCHAR(50),
    Course VARCHAR(50)
);

INSERT INTO Students_Courses (StudentID, StudentName, Course)
VALUES
(1, 'John Doe', 'Math'),
(1, 'John Doe', 'Science'),
(2, 'Jane Smith', 'Math'),
(3, 'Sam Brown', 'Science'),
(3, 'Sam Brown', 'English'),
(3, 'Sam Brown', 'Math');
```

### Step 3: Second Normal Form (2NF)

A table is in Second Normal Form (2NF) if:
- It is already in 1NF.
- All non-key attributes are fully functionally dependent on the primary key.

In simpler terms, every non-key column must depend on the entire primary key, not just part of it.

#### Example:
In our current table, `StudentName` depends only on `StudentID`, not on `Course`. To achieve 2NF, we split the table into two:

1. A `Students` table that stores student information.
2. A `Courses` table that stores which students are enrolled in which courses.

##### Students Table:
| StudentID | StudentName |
|-----------|-------------|
| 1         | John Doe    |
| 2         | Jane Smith  |
| 3         | Sam Brown   |

##### Courses Table:
| StudentID | Course  |
|-----------|---------|
| 1         | Math    |
| 1         | Science |
| 2         | Math    |
| 3         | Science |
| 3         | English |
| 3         | Math    |

#### SQL Queries for 2NF:
```sql
CREATE TABLE Students (
    StudentID INT PRIMARY KEY,
    StudentName VARCHAR(50)
);

CREATE TABLE Courses (
    StudentID INT,
    Course VARCHAR(50),
    FOREIGN KEY (StudentID) REFERENCES Students(StudentID)
);

INSERT INTO Students (StudentID, StudentName)
VALUES
(1, 'John Doe'),
(2, 'Jane Smith'),
(3, 'Sam Brown');

INSERT INTO Courses (StudentID, Course)
VALUES
(1, 'Math'),
(1, 'Science'),
(2, 'Math'),
(3, 'Science'),
(3, 'English'),
(3, 'Math');
```

### Step 4: Third Normal Form (3NF)

A table is in Third Normal Form (3NF) if:
- It is in 2NF.
- There are no transitive dependencies, meaning no non-key attribute depends on another non-key attribute.

#### Example:
Imagine we add a column `CourseInstructor` in the `Courses` table, which stores the instructor for each course:

| StudentID | Course  | CourseInstructor |
|-----------|---------|------------------|
| 1         | Math    | Dr. Smith        |
| 1         | Science | Dr. Jones        |
| 2         | Math    | Dr. Smith        |
| 3         | Science | Dr. Jones        |
| 3         | English | Dr. Clark        |
| 3         | Math    | Dr. Smith        |

Here, `CourseInstructor` depends on `Course`, not on `StudentID`. To achieve 3NF, we should split this into two tables:

1. A `Courses` table with course details.
2. An `Enrollments` table that links students to courses.

##### Courses Table:
| CourseID | Course  | CourseInstructor |
|----------|---------|------------------|
| 1        | Math    | Dr. Smith        |
| 2        | Science | Dr. Jones        |
| 3        | English | Dr. Clark        |

##### Enrollments Table:
| StudentID | CourseID |
|-----------|----------|
| 1         | 1        |
| 1         | 2        |
| 2         | 1        |
| 3         | 2        |
| 3         | 3        |
| 3         | 1        |

#### SQL Queries for 3NF:
```sql
CREATE TABLE Courses (
    CourseID INT PRIMARY KEY,
    Course VARCHAR(50),
    CourseInstructor VARCHAR(50)
);

CREATE TABLE Enrollments (
    StudentID INT,
    CourseID INT,
    FOREIGN KEY (StudentID) REFERENCES Students(StudentID),
    FOREIGN KEY (CourseID) REFERENCES Courses(CourseID)
);

INSERT INTO Courses (CourseID, Course, CourseInstructor)
VALUES
(1, 'Math', 'Dr. Smith'),
(2, 'Science', 'Dr. Jones'),
(3, 'English', 'Dr. Clark');

INSERT INTO Enrollments (StudentID, CourseID)
VALUES
(1, 1),
(1, 2),
(2, 1),
(3, 2),
(3, 3),
(3, 1);
```

### Step 5: Boyce-Codd Normal Form (BCNF)

BCNF is a stronger version of 3NF. A table is in BCNF if:
- It is in 3NF.
- For every functional dependency (A → B), A should be a super key.

#### Example:
Consider a situation where a professor can only teach one subject at a time, but a subject can be taught by multiple professors:

| ProfessorID | Course  | RoomNo |
|-------------|---------|--------|
| 1           | Math    | 101    |
| 1           | Science | 102    |
| 2           | Math    | 101    |
| 3           | English | 103    |

Here, `RoomNo` depends on both `ProfessorID` and `Course`. To convert this to BCNF, we split it into two tables:

1. A `Professors` table that assigns rooms to professors.
2. A `Teaches` table that shows which courses professors teach.

##### Professors Table:
| ProfessorID | RoomNo |
|-------------|--------|
| 1           | 101    |
| 1           | 102    |
| 2           | 101    |
| 3           | 103    |

##### Teaches Table:
| ProfessorID | Course  |
|-------------|---------|
| 1           | Math    |
| 1           | Science |
| 2           | Math    |
| 3           | English |

#### SQL Queries for BCNF:
```sql
CREATE TABLE Professors (
    ProfessorID INT,
    RoomNo INT
);

CREATE TABLE Teaches (
    ProfessorID INT,
    Course VARCHAR(50),
    FOREIGN KEY (ProfessorID) REFERENCES Professors(ProfessorID)
);

INSERT INTO Professors (ProfessorID, RoomNo)
VALUES
(1, 101),
(1, 102),
(2, 101),
(3, 103);

INSERT INTO Teaches (ProfessorID, Course)
VALUES
(1, 'Math'),
(1, 'Science'),
(2, 'Math'),
(3, 'English');
```

